### Agent-Lab: Adaptive RAG Agent

---

In [1]:
import os
os.chdir('..')

# third party dependencies
from langgraph.graph import END, StateGraph, START

# local dependencies
from notebooks import experiment_utils
from app.core.container import Container
from app.services.agent_types.adaptive_rag.agent import (
    AgentState, 
    AdaptiveRagAgent
)

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

Overriding of current TracerProvider is not allowed
/home/bsantanna/miniconda3/envs/dev/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [2]:
# create ollama agent
# agent_id = experiment_utils.create_ollama_agent(agent_type="adaptive_rag")
agent_id = '4c83fbb4-c4b8-475b-9a9d-031da527a5d9'

In [5]:
# get agent instance
adaptive_rag_agent = container.adaptive_rag_agent()

# Create Graph
workflow_builder = adaptive_rag_agent.get_workflow_builder(agent_id)

In [4]:
# Nodes definition
# workflow_builder.add_node("retrieve", adaptive_rag_agent.retrieve)
# workflow_builder.add_node("grade_documents", adaptive_rag_agent.grade_documents)
# workflow_builder.add_node("generate", adaptive_rag_agent.generate)
# workflow_builder.add_node("transform_query", adaptive_rag_agent.transform_query)